In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

device='cuda' if torch.cuda.is_available() else 'cpu'

Data Preprocessing

In [ ]:
df_train=pd.read_csv('train (1).csv')

In [ ]:
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
pd.set_option('display.max_columns',None)
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
pd.set_option('display.max_rows',None)
df_train.isna().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [ ]:
df_train=df_train.drop('PoolQC',axis=1)
df_train=df_train.drop('Fence',axis=1)
df_train=df_train.drop('MiscFeature',axis=1)
df_train=df_train.drop('FireplaceQu',axis=1)
df_train=df_train.drop('Alley',axis=1)

In [ ]:
df_train=df_train.dropna()

In [ ]:
dummy_train_df=pd.get_dummies(df_train)

In [ ]:
dummy_train_df['total_sf']=dummy_train_df['1stFlrSF']+dummy_train_df['2ndFlrSF']+dummy_train_df['TotalBsmtSF']
dummy_train_df['total_bath']=dummy_train_df['BsmtFullBath']+dummy_train_df['BsmtHalfBath']+dummy_train_df['FullBath']+dummy_train_df['HalfBath']

Analysis

In [ ]:
correlation=dummy_train_df.corr()
correlation=correlation[correlation>0].stack()
correlation['SalePrice'].describe()
#only taking feature that it correlation is more than median
position=correlation.loc['SalePrice'][correlation.loc['SalePrice']>0.6].index
filtered_feature=dummy_train_df[position]
filtered_feature=filtered_feature.drop('SalePrice',axis=1)

In [ ]:
SalePrice=dummy_train_df['SalePrice']
filtered_feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1459
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   OverallQual  1094 non-null   int64
 1   TotalBsmtSF  1094 non-null   int64
 2   1stFlrSF     1094 non-null   int64
 3   GrLivArea    1094 non-null   int64
 4   GarageCars   1094 non-null   int64
 5   GarageArea   1094 non-null   int64
 6   total_sf     1094 non-null   int64
 7   total_bath   1094 non-null   int64
dtypes: int64(8)
memory usage: 76.9 KB


Transforming data into tensor

In [ ]:
scaler = MinMaxScaler()
scaler_y=MinMaxScaler()

In [ ]:
x=torch.from_numpy(np.array(filtered_feature)).float()
y=torch.from_numpy(np.array(SalePrice)).float()
x= scaler.fit_transform(x)
y= scaler_y.fit_transform(y.reshape(-1, 1))
x_train, samples, y_train, labels = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, y_train, samples=torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float(), torch.from_numpy(samples).float()
x_train

tensor([[0.7500, 0.2633, 0.2953,  ..., 0.3784, 0.2406, 0.4000],
        [0.5000, 0.1812, 0.2551,  ..., 0.2528, 0.1803, 0.4000],
        [0.5000, 0.1222, 0.1001,  ..., 0.1892, 0.1546, 0.6000],
        ...,
        [0.5000, 0.1624, 0.2863,  ..., 0.2925, 0.1821, 0.2000],
        [0.7500, 0.1647, 0.1634,  ..., 0.3831, 0.2583, 0.4000],
        [0.6250, 0.0998, 0.0992,  ..., 0.0588, 0.1397, 0.2000]])

In [ ]:
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
!pip install joblib

Model

In [ ]:
class nn(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.model=torch.nn.Sequential(
      torch.nn.Linear(8,256),
      torch.nn.ReLU(),
      torch.nn.Linear(256,516),
      torch.nn.ReLU(),
      torch.nn.Linear(516,256),
      torch.nn.ReLU(),
      torch.nn.Linear(256,128),
      torch.nn.ReLU(),
      torch.nn.Linear(128,1),
    )

  def forward(self,x):
    return(self.model(x))

In [ ]:
cpu_seed = torch.initial_seed()
print(f"Random Seed (CPU): {cpu_seed}")

Random Seed (CPU): 3047280439319536781


In [ ]:
model=nn()

Training

In [ ]:
class spaced_learning():
    def __init__(self, n_nn=50):
        self.n_nn = n_nn

    def fit(self, X, y):
        self.models = []
        ensemble_weights = []
        base_acc=100
        for _ in range(self.n_nn):
            print(_)
            x_samp, y_samp = self._bootstraped(X, y)

            model_copy = self._train(DataLoader(TensorDataset(x_samp, y_samp), batch_size=32))

            self.models.append(model_copy)

    def _train(self, X):
        model_copy = model.to(device)
        optim = torch.optim.SGD(params=model_copy.parameters(), lr=0.01)
        loss_fn = torch.nn.MSELoss()
        epochs =60
        for epoch in range(epochs):
            for batch, (X_batch, y_batch) in enumerate(X):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                model_copy.train()
                ypred = model_copy(X_batch)
                loss=loss_fn(ypred , y_batch)
                print(f"loss{loss}||epoch{epoch}||batch{batch}")
                optim.zero_grad()
                loss.backward()
                optim.step()
        return model_copy

    def _bootstraped(self, X, y):
        n_samps = X.shape[0]
        idx = np.random.choice(n_samps, 1000, replace=True)
        return X[idx], y[idx]

    def _voting(self, predictions):
        if isinstance(predictions[0], np.ndarray):
            average_predictions = np.mean(predictions, axis=0)
            return average_predictions
        else:
            return np.mean(predictions, axis=0)

    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32)
        predictions = []
        for i, model in enumerate(self.models):
            model.eval()
            with torch.no_grad():
                ypred = model(X)
                ypred = ypred.cpu().numpy()
                predictions.append(ypred)

        ensemble_predictions = self._voting(predictions)
        return ensemble_predictions


In [ ]:
model0=spaced_learning()
print(x_train.dtype,y_train.dtype)
model0.fit(x_train,y_train)

Streaming output truncated to the last 5000 lines.
loss0.0011061965487897396||epoch23||batch26
loss0.0009845730382949114||epoch23||batch27
loss0.0010955516481772065||epoch23||batch28
loss0.0011751896236091852||epoch23||batch29
loss0.0009405456949025393||epoch23||batch30
loss0.00027618464082479477||epoch23||batch31
loss0.0009853318333625793||epoch24||batch0
loss0.0007829670794308186||epoch24||batch1
loss0.0022524071391671896||epoch24||batch2
loss0.0012653369922190905||epoch24||batch3
loss0.0012647057883441448||epoch24||batch4
loss0.0005101807764731348||epoch24||batch5
loss0.001347168581560254||epoch24||batch6
loss0.0005652426625601947||epoch24||batch7
loss0.002424790058284998||epoch24||batch8
loss0.0010827837977558374||epoch24||batch9
loss0.000766271841712296||epoch24||batch10
loss0.0008860004018060863||epoch24||batch11
loss0.0032067596912384033||epoch24||batch12
loss0.0016256531234830618||epoch24||batch13
loss0.0012405095621943474||epoch24||batch14
loss0.0014752378920093179||epoch24||b

Testing

In [ ]:
test=pd.read_csv('test (1).csv')
test['total_sf']=test['1stFlrSF']+test['2ndFlrSF']+test['TotalBsmtSF']
test['total_bath']=test['BsmtFullBath']+test['BsmtHalfBath']+test['FullBath']+test['HalfBath']
feats=pd.get_dummies(test[['OverallQual','TotalBsmtSF','1stFlrSF','GrLivArea','GarageCars','GarageArea','total_sf','total_bath']])
print(feats.head)
feats=torch.from_numpy(scaler.transform(feats)).float()

<bound method NDFrame.head of       OverallQual  TotalBsmtSF  1stFlrSF  GrLivArea  GarageCars  GarageArea  \
0               5        882.0       896        896         1.0       730.0   
1               6       1329.0      1329       1329         1.0       312.0   
2               5        928.0       928       1629         2.0       482.0   
3               6        926.0       926       1604         2.0       470.0   
4               8       1280.0      1280       1280         2.0       506.0   
5               6        763.0       763       1655         2.0       440.0   
6               6       1168.0      1187       1187         2.0       420.0   
7               6        789.0       789       1465         2.0       393.0   
8               7       1300.0      1341       1341         2.0       506.0   
9               4        882.0       882        882         2.0       525.0   
10              7       1405.0      1337       1337         2.0       511.0   
11              6     

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [ ]:
feat[1459]

IndexError: ignored

In [ ]:
pred=model0.predict(feats.to(device))

<ipython-input-268-299608bfa78c>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


In [ ]:
model.state_dict

In [ ]:
pred=scaler_y.inverse_transform(pred)

In [ ]:
pred

array([[114381.14 ],
       [157449.47 ],
       [158684.94 ],
       ...,
       [149737.36 ],
       [104331.266],
       [218015.73 ]], dtype=float32)

In [ ]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': pred.squeeze()})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
PATH = "model.pt"

# Save
torch.save(model.state_dict(), PATH)

In [ ]:
print('__pyTorch VERSION:', torch.__version__)

In [ ]:
model.state_dict()